# Course 4: Week 1

## # Convolution

[What is an intuitive explanation for convolution?](https://www.quora.com/What-is-an-intuitive-explanation-for-convolution)

[Intuitively Understanding Convolutions for Deep Learning](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1)

## ## Edge detection

We'll apply a filter, running over the image matrix.

The following filter (kernel) is used for vertical edge detection:

![](media/ed.png)

(For each iteration, this could be represented as: $line_{from\_picture} * column_{from\_filter}.T$)

Why it works:

![](media/ed2.png)

[...] "a vertical edge is where there are bright pixels on the left, you do not care that much what is in the middle and dark pixels on the right."

![](media/ed3.png)

___
**Example:**

![](media/ed4.png)

___
**Example:**

![](media/ed5.png)

## ## Padding

Default convolving will lead to lose information on the image's border/edges.

![](media/padd.png)

So, we could add/use padding. Which will increase the relevance of the image's borders/edges pixels and preserve the output dimmension equals to the input's.

![](media/padd2.png)

___
Forms of implementing padding:

![](media/padd3.png)

$padding = \frac{f - 1}{2}$

## ## Strided convolutions

Works as a `step`, $(i,j)$-wise.

With a padding of $p$ and a stride of $s$:


![](media/stri.png)

$i.e.:$

______

![](media/stri3.png)

___

(For non-integer values, round down (floor) the value.)

The kernel product only happens when the multiplication is possible:

![](media/stri2.png)

___ 
**Math:**

He then explains about this twist and turn,

![](media/stri4.png)

**Disclaimer:**

![](media/stri5.png)

Achieving this result is just a matter of performin the following:

In [131]:
import numpy as np

v = np.array([[3, 4, 5], [1, 0, 2], [-1, 9, 7]])

print('input:\n{}'.format(v))

# flipped identity matrix
anti_id = np.array([[0, 0, 1],[0, 1, 0],[1, 0, 0]])

# flip horizontally: V * anti_id
first_step = np.dot(v, anti_id)

# flip vertically: anti_id * v
second_step = np.dot(anti_id, first_step)

print('\noutput:\n{}'.format(second_step))

input:
[[ 3  4  5]
 [ 1  0  2]
 [-1  9  7]]

output:
[[ 7  9 -1]
 [ 2  0  1]
 [ 5  4  3]]


### ### Convolutions over volume (RGB)


You'll apply convolutions, likewise, on each layer of the rgb channels.

![](media/rgb.png)

We could apply filters that only work on a single channel, zeroing the other layers.

Convolving on volumes will allow us to operate on RGB pictures and detect more complex edges, like horizontal edges, or angled edges.

![](media/rgb2.png)

![](media/rgb3.png)

![](media/rgb4.png)

![](media/rgb5.png)

![](media/rgb6.png)

![](media/rgb7.png)

- If layer '$l$' is a convolution layer:

$f^{[l]}$ = filter size

$p^{[l]}$ = padding

$s^{[l]}$ = stride


- The input of layer '$l$' has dimension:

$n_{height}^{[l-1]}$ x $n_{width}^{[l-1]}$ x $n_{c}^{[l-1]}$

In [139]:
!mv -v /home/f4119597/Downloads/Screenshot.png media/rgb8.png

mv: cannot stat '/home/f4119597/Downloads/Screenshot.png': No such file or directory
